# Lab: Importing and Exporting Geography Data with BigQuery
BigQuery is a serverless data warehouse solution on Google Cloud Platform (GCP) that allows users to interact with their data using standard ANSI SQL.

Using the BigQuery Python API, BigQuery databases can be directly accessed via Python and/or Jupyter notebooks.

BigQuery GIS allows point, line or polygon geospatial data to be stored as a special `GEOGRAPHY` data type within a BigQuery table.  This `GEOGRAPHY` datatype is stored as a [well-known text](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) object.  

## Prerequisites
* Basic understanding of relational databases and ANSI SQL
* Basic understanding of Python
 
## Learning Objectives
When you complete this lab, you should be able to:
* Interact with BigQuery datasets within Jupyter notebooks
* Convert geospatial data into a BigQuery GIS `GEOGRAPHY` data object
* Export BigQuery GIS `GEOGRAPHY` objects into other formats

## Getting Started
1. First, let's create a dataset to store the model.  [Click here to open the BigQuery UI.](https://console.cloud.google.com/bigquery?_ga=2.44329517.868398427.1595536877-1775816527.1575499949)

2. In the BigQuery Web UI, select `Create Dataset`   
![create dataset](./images/create_dataset.jpg)

3. Give the dataset a name that you can reference later and keep the default settings.  I used `gis_example`.

4. Click the `Create Table` button  
![create table](./images/create_table.jpg)

5. Create a new table called `nyc_landmarks` with the schema as seen below  
![table schema](./images/nyc_landmarks.jpg)

In order to connect to BigQuery in Python, you need to install the BigQuery Python client library and the BigQuery Storage API.  You will also need to set up the notebook to use the "magic" command, which we will be using to access BigQuery throughout this lab.  

In addition, you will need to install the `pandas` package if you don't have it already installed.

Note: If you are running this notebook in a GCP environment, these packages should be pre-installed.

In [ ]:
%pip install --upgrade 'google-cloud-bigquery'
%pip install --upgrade 'google-cloud-bigquery-storage'
%pip install --upgrade 'pyarrow'
%pip install --upgrade 'pandas'

%load_ext google.cloud.bigquery

## Creating `GEOGRAPHY` Points
Now that we have a table in BigQuery, we can start to populate it with some data.

For our first landmark, we will insert the Empire State Building into the table, which has the following coordinates:
* Latitude: 40.7484
* Longitude: -73.9857

`GEOGRAPHY` objects can be created from the following data formats:
* Coordinate (latitude/longitude pairs)
* Well-Known Text (WKT) objects
* GeoJSON

We can directly write ANSI SQL to query BigQuery tables by using the `%%bigquery` [magic command](https://googleapis.dev/python/bigquery/latest/magics.html).

In [1]:
%%bigquery
INSERT INTO `gis_example.nyc_landmarks` (name,latitude,longitude)
values ('Empire State Building',40.7484,-73.9857);

""


Now that we have our first row of data in our table, we can use a "constructor" function to convert the latitude/longitude coordinates into a `GEOGRAPHY` data object.  The [ST_GEOGPOINT](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_geogpoint) function takes coordinate values and converts them to the special `GEOGRAPHY` data type.  

This function creates a point data object, because it describes a single geospatial point.

In [2]:
%%bigquery
UPDATE `gis_example.nyc_landmarks`
SET landmark_geom = ST_GEOGPOINT(longitude,latitude)
WHERE name = 'Empire State Building';

""


We can now see what we have entered into the table, which includes the newly constructed `GEOGRAPHY` point.

In [3]:
%%bigquery
SELECT *
FROM `gis_example.nyc_landmarks`;

,name,latitude,longitude,landmark_geom
0,Empire State Building,40.7484,-73.9857,POINT(-73.9857 40.7484)


The `GEOGRAPHY` data type is stored in [Well-Known Text (WKT)](https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry) format.

Let's go ahead and add a few more landmarks to our table.

In [4]:
%%bigquery
INSERT INTO `gis_example.nyc_landmarks` (name,latitude,longitude)
values 
('Rockefeller Center',40.7587,-73.9787),
('New York Stock Exchange',40.7069,-74.0113),
('Statue of Liberty',40.6892,-74.0445);

SELECT *
FROM `gis_example.nyc_landmarks`;

,name,latitude,longitude,landmark_geom
0,Rockefeller Center,40.7587,-73.9787,None
1,Statue of Liberty,40.6892,-74.0445,None
2,New York Stock Exchange,40.7069,-74.0113,None
3,Empire State Building,40.7484,-73.9857,POINT(-73.9857 40.7484)


**Try It:**  
Can you update the `landmark_geom` field for the new landmarks added in the above query?  
*Hint:* Use the `ST_GEOGPOINT` constructor function.

In [ ]:
%%bigquery
#Modify the query in this code cell to update the landmark_geo field with new GEOGRAPHY objects
UPDATE `gis_example.nyc_landmarks`
SET landmark_geom = ??
WHERE name <> 'Empire State Building';

SELECT *
FROM `gis_example.nyc_landmarks`;

## Creating `GEOGRAPHY` Linestrings
Now that we have several `GEOGRAPHY` point objects, we can create a linestring `GEOGRAPHY` objects that connects our points together.  

This can be done using the [ST_MAKELINE](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_makeline) constructor function.

Your output should look like this: `LINESTRING(-73.9857 40.7484, -73.9787 40.7587)`

In [6]:
%%bigquery
#Create line that connects the Empire State Building and Rockefeller Center GEOGRAPHY points
SELECT 
ST_MAKELINE(
  (SELECT landmark_geom FROM `gis_example.nyc_landmarks` WHERE name = 'Empire State Building'), #Subquery #1: Get GEOGRAPHY point for Empire State Building
  (SELECT landmark_geom FROM `gis_example.nyc_landmarks` WHERE name = 'Rockefeller Center')     #Subquery #2: Get GEOGRAPHY point for Rockefeller Center
) landmark_path;

,landmark_path
0,"LINESTRING(-73.9857 40.7484, -73.9787 40.7587)"


**Try It:**  
Using subqueries like those above, can you create a `GEOGRAPHY` linestring object that connects Rockefeller Center to the New York Stock Exchange?

In [ ]:
%%bigquery
#Create line that connects the Rockefeller Center and New York Stock Exchange GEOGRAPHY points
SELECT 
ST_MAKELINE(
  (??), #Subquery #1: Get GEOGRAPHY point for Rockefeller Center
  (??)  #Subquery #2: Get GEOGRAPHY point for New York Stock Exchange
) landmark_path;

## Creating `GEOGRAPHY` Polygons
The `GEOGRAPHY` data type also supports polygon objects.  A geospatial polygon is a closed data shape that can be built using the [ST_MAKEPOLYGON](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_makepolygon) constructor function.  This constructor function takes a linestring as its input, where the linestring has to be closed (i.e., the beginning and endpoint of the linestring must be the same).  If the start and end points of the linestring are not the same, the constructor will automatically construct a final edge that connects the starting point to the end point.

Let's create a polygon that has three vertices:
* Empire State Building
* Rockefeller Center
* New York Stock Exchange

We can make a linestring that connects all three landmark points by using an array as the input to the `ST_MAKELINE` constructor function.

This linestring can then be given to the `ST_MAKEPOLYGON` constructor as its argument.

You output should look like this: `POLYGON((-74.0113 40.7069, -73.9787 40.7587, -73.9857 40.7484, -74.0113 40.7069))`

In [7]:
%%bigquery
SELECT ST_MAKEPOLYGON(landmark_path.path) landmark_polygon
FROM (
  SELECT ST_MAKELINE(ARRAY(SELECT landmark_geom FROM `gis_example.nyc_landmarks` 
                           WHERE name in ('Empire State Building','Rockefeller Center','New York Stock Exchange'))) path) landmark_path;

,landmark_polygon
0,"POLYGON((-73.9857 40.7484, -74.0113 40.7069, -..."


## Exporting `GEOGRAPHY` Objects
We are now going to look at "formatter" functions, which allow us to export `GEOGRAPHY` data types into other formats.

Let's export the `GEOGRAPHY` point data for the Statue of Liberty into [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) format.

You output should look like this: `{ "type": "Point", "coordinates": [-74.0445, 40.6892] }`

In [8]:
%%bigquery
SELECT ST_ASGEOJSON(landmark_geom) GeoJSON
FROM `gis_example.nyc_landmarks` 
WHERE name = 'Statue of Liberty';

,GeoJSON
0,"{ ""type"": ""Point"", ""coordinates"": [-74.0445, 4..."


We can use the [ST_ASGEOJSON](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_asgeojson) formatter function to convert all of the `GEOGRAPHY` data objects in the `nyc_landmarks` table to GeoJSON format.

The "export_geo" tag after the `%%bigquery` "magic" command saves the query output into a `pandas` dataframe.

In [9]:
%%bigquery export_geo
SELECT name, ST_ASGEOJSON(landmark_geom) GeoJSON
FROM `gis_example.nyc_landmarks`;

In [10]:
#View pandas dataframe with landmark name and GeoJSON point coordinates
export_geo.head()

,name,GeoJSON
0,Rockefeller Center,"{ ""type"": ""Point"", ""coordinates"": [-73.9787, 4..."
1,Statue of Liberty,"{ ""type"": ""Point"", ""coordinates"": [-74.0445, 4..."
2,New York Stock Exchange,"{ ""type"": ""Point"", ""coordinates"": [-74.0113, 4..."
3,Empire State Building,"{ ""type"": ""Point"", ""coordinates"": [-73.9857, 4..."


**Try It:**  
Can you export the `GEOGRAPHY` point data for the New York Stock Exchange formatted as well-known text (WKT)?  
*Hint:* Use the [ST_ASTEXT](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions#st_astext) formatter function.

In [ ]:
%%bigquery
SELECT ?? 
FROM `gis_example.nyc_landmarks` 
WHERE name = 'New York Stock Exchange';

## Explore Further
Now that you have gotten started with `GEOGRAPHY` data types in BigQuery GIS, you can view the full documentation for `GEOGRAPHY` functions [here](https://cloud.google.com/bigquery/docs/reference/standard-sql/geography_functions).